In [15]:
#generic linear programming
import json
from ortools.linear_solver import pywraplp
Obj_Func = ""
sign = 3
ans_map = {}
num_of_var = 0
splitted_objective_function =""

def getConstrExpr(constraint):
    if "<=" in constraint:
        splitted_constraint = constraint.split("<=")

    elif ">=" in constraint:
        splitted_constraint = constraint.split(">=")

    lhs = splitted_constraint[0]
    rhs = splitted_constraint[1]

    splitted_lhs = []
    if "+" in lhs:
        splitted_lhs = lhs.split("+")

    elif "-" in lhs:
        splitted_lhs = lhs.split("-")
    
    else:
        splitted_lhs.append(lhs)

    #print("splitted lhs:",splitted_lhs)
    
    num_of_var = len(splitted_lhs)
    
    new_constr = ""
    for nv in range(0,num_of_var):
        coeff_index = splitted_lhs[nv].split('x')
        coeff = coeff_index[0]
        index = coeff_index[1]
        if coeff=="":
            coeff = "1"
        new_constr+=str(coeff)+" * x["+str(index)+"]"
        if nv!=num_of_var-1:
            if "+" in lhs:
                new_constr+=" + "
            elif "-" in lhs:
                new_constr+=" - "
          
    if "<=" in constraint:
        new_constr+=" <= "+str(rhs)

    elif ">=" in constraint:
        new_constr+=" >= "+str(rhs)
    return new_constr
  
    
def getExpr(splitted_objective_function,num_of_var):
    obj_func = ""
    for nv in range(0,num_of_var):
        coeff_index = splitted_objective_function[nv].split('x')
        coeff = coeff_index[0]
        index = coeff_index[1]
        if coeff=="":
            coeff = "1"
        obj_func+=str(coeff)+" * x["+str(index)+"]"
        if nv!=num_of_var-1:
            #if sign==1:
            #    obj_func+=" + "
            #elif sign==0:
            #    obj_func+=" - "
            obj_func+=" + "
    
    return obj_func

def main(ObjectiveFunction, ConstraintList):
    # [START solver]
    # Create the linear solver with the GLOP backend.
    solver = pywraplp.Solver.CreateSolver('GLOP')
    # [END solver]

    # [START variables]
    infinity = solver.infinity()
    
    # Define the decision variables
    #command = "Maximize 8x1+5x2"
    #command = input("Enter command\n")
    command = ObjectiveFunction
    splitted_command = command.split()

    ans_map['Objective_Function'] = command

    todo = splitted_command[0]

    if "+" in splitted_command[1]:
        sign = 1
        splitted_objective_function = splitted_command[1].split("+")

    elif "-" in splitted_command[1]:
        sign = 0
        splitted_objective_function = splitted_command[1].split("-")

    num_of_var = len(splitted_objective_function)
    
    x = {i: solver.NumVar(0.0, infinity, f'x{i}') for i in range(1, num_of_var+1)}

    print('Number of variables =', solver.NumVariables())
    ans_map['NumOfVar'] = solver.NumVariables()
    # [END variables]

    # [START constraints]
    # 2x+y<=1000
    #constr_list = ["2x1+x2<=1000","3x1+4x2<=2400","x1+x2<=700","x1-x2<=350","x1>=0","x2>=0"]
    
    #constr_input = input("Enter constraints\n")
    constr_input = ConstraintList
    constr_list = constr_input.split(",")
    for constr in constr_list:
        constr = getConstrExpr(constr)
        constr = eval(constr)
        solver.Add(constr)

    ans_map['Constraints'] = constr_input

    # x >=0
    #solver.Add(x[1] >= 0.0)

    # y >=0
    #solver.Add(x[2] >= 0.0)

    print('Number of constraints =', solver.NumConstraints())
    ans_map['NumOfConstraints'] = solver.NumConstraints()

    # [END constraints]
    # [START objective]
    # Maximize 8x+5y
    objFunc = getExpr(splitted_objective_function,num_of_var)
    #converting string into exp
    if todo=="Maximize":
        #solver.Maximize(8 * x[1] + 5 * x[2])
        objFunc = eval(objFunc)
        solver.Maximize(objFunc)
    # [END objective]

    # [START solve]
    status = solver.Solve()

    # [END solve]
    solution = {}
    # [START print_solution]
    if status == pywraplp.Solver.OPTIMAL:
        solution['exists']= "yes"
        print('Solution:')
        print('Objective value =', solver.Objective().Value())
        solution['ObjectiveValue'] = solver.Objective().Value()
        
        #generic 
        for var in range(1, num_of_var+1):
            print('x[',var,'] = ', x[var].solution_value())
            key = 'x'+str(var)+'_sol_val'
            solution[key] = x[var].solution_value()

        print("offset ", solver.Objective().BestBound())
        
    else:
        print('The problem does not have an optimal solution.')
        solution['exists']="no"

    ans_map['solution'] = solution

    # [END print_solution]
    print("cons",solver.constraints()[2].ub())
    # x=round(x.solution_value())
    constraint_compute=solver.ComputeConstraintActivities()

    #Slack: RHS – LHS value in a  ≤ constraint
    #The amount of resource that is not used
    compute_slack=[]
    for i in range(len(solver.constraints())):
        # print("poss ", solver.constraints()[i].ub()-round(constraint_compute[i]))
        if solver.constraints()[i].ub()==infinity:
            print("inf")
        else:
            compute_slack.append( {"constraint":solver.constraints()[i],"slack": solver.constraints()[i].ub()-round(constraint_compute[i])})
    # [START advanced]
    print('\nAdvanced usage:')
    print('Problem solved in %f milliseconds' % solver.wall_time())
    ans_map['solving_time']=solver.wall_time()
    print('Problem solved in %d iterations' % solver.iterations())
    ans_map['iterations']=solver.iterations()
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
    ans_map['branch_bound_nodes']=solver.nodes()
    # [END advanced]

    ############# slack/surplus  --------------------------------

    #print("compute slack: ",compute_slack)
    track_index = 1
    
    constraint_details={}
    
    for val in compute_slack:
        print("*********G: ")
        build_constraint = ""
        for var in range(1, num_of_var+1):
            coefficient = int(val['constraint'].GetCoefficient(x[var]))
            if coefficient!=1:
                build_constraint += str(coefficient)+"x"+str(var)
            else:
                build_constraint += "x"+str(var)
                
            if var!=num_of_var:
                build_constraint+=" + "
        
        print(build_constraint)
            
        
        constraint_map={}
        if val['slack'] ==0.0:
            print("This constraint ", str(build_constraint)," is a binding constraint & has a direct impact on the optimal solution")
            constraint_map['binding'] = 1
            #get_constraint = val['constraint'].GetCoefficient(x[1]),"* x1
            #shadow price
            print("shadow price is ", val['constraint'].dual_value())
            constraint_map['shadow_price'] = val['constraint'].dual_value()
        else:
            print("This constraint ",str(build_constraint)," is a non-binding constraint & has no direct impact on the optimal solution")
            constraint_map['binding'] = 0
            #dual
            print("dual ", val['constraint'].dual_value())
            constraint_map['dual']=val['constraint'].dual_value()
        
            if  val['slack'] >0.0:
                print("The total limit is not used yet. It had total limit of", val['constraint'].ub(), "but used only",  val['constraint'].ub()-val['slack'])
                #ans_map[]
                constraint_map['total_limit']=val['constraint'].ub()
                constraint_map['used']=val['constraint'].ub()-val['slack']
            else:
                print("The total limit is not used yet. It had total limit of", val['constraint'].ub(), "but used only",  val['constraint'].ub()+val['slack'])
        
        constraint_details[build_constraint]=constraint_map
        
    #print("the reduced cost of x1 is", x[1].reduced_cost(), x[2].basis_status())
    #print("the reduced cost of y is", y.reduced_cost(), x.basis_status())
    ans_map['constraint_details']=constraint_details
    return ans_map

def checkSign(val):
    if val <0:
        return True
    else:
        return False

#if __name__ == '__main__':
#    main()
# [END program]

#Maximize 8x1+5x2
#2x1+x2<=1000,3x1+4x2<=2400,x1+x2<=700,x1-x2<=350,x1>=0,x2>=0
if __name__ == '__main__':
    #objfunc = input("Enter command\n")
    objfunc = "Maximize 8x1+5x2"
    #constr = input("Enter constraints\n")
    constr="2x1+x2<=1000,3x1+4x2<=2400,x1+x2<=700,x1-x2<=350,x1>=0,x2>=0"
    ans = main(objfunc, constr)
    print(json.dumps(ans, indent=1))
 

Number of variables = 2
Number of constraints = 6
Solution:
Objective value = 4360.0
x[ 1 ] =  319.99999999999983
x[ 2 ] =  360.00000000000017
offset  -inf
cons 700.0
inf
inf

Advanced usage:
Problem solved in 6.000000 milliseconds
Problem solved in 4 iterations
Problem solved in -1 branch-and-bound nodes
*********G: 
2x1 + x2
This constraint  2x1 + x2  is a binding constraint & has a direct impact on the optimal solution
shadow price is  3.399999999999999
*********G: 
3x1 + 4x2
This constraint  3x1 + 4x2  is a binding constraint & has a direct impact on the optimal solution
shadow price is  0.4000000000000004
*********G: 
x1 + x2
This constraint  x1 + x2  is a non-binding constraint & has no direct impact on the optimal solution
dual  -0.0
The total limit is not used yet. It had total limit of 700.0 but used only 680.0
*********G: 
x1 + -1x2
This constraint  x1 + -1x2  is a non-binding constraint & has no direct impact on the optimal solution
dual  -0.0
The total limit is not used yet